In [104]:
%pip install pathway transformers

  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
  Using cached pydantic_core-2.18.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.5 kB)
Using cached pydantic-2.7.4-py3-none-any.whl (409 kB)
Using cached pydantic_core-2.18.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 1.6.3 requires pydantic<3.0.0,>=2.8.0, but you have pydantic 2.7.4 which is incompatible.


In [105]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "sshleifer/distilbart-cnn-12-6"  # Smaller, distilled version of BART
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [106]:
#Now we create our documnet retrieval pipeline.
#This will give us more relevant data than what our llm model has
class Retrieval_Pipeline:
    def __init__(self,documents):
        self.documents = documents #Store the documents over here

    def retrieve(self,query):#Given a query we would want to retrieve relavant documents
        results = list()

        for doc in self.documents:
            if query.lower() in doc.lower():
                results.append(doc)
        return results


In [107]:
#An example of how this retrieval process works

documents = ["NJACK is the coding club of IIT Patna",
             "IIT Patna is an institute of National importance",
             "NJACK has 4 divisions",
             "NJACK is a technical club",
             "There are many technical clubs at IIT Patna",
             "NJACK has sub coordinators for every division ",
             "NJACK has cp,ML,devos and cybersec divsions",
             "Bihar is in Patna",
             "There are many technical clubs at IIT Patna"]

pipeline = Retrieval_Pipeline(documents)

print(pipeline.retrieve('NJACK'))


['NJACK is the coding club of IIT Patna', 'NJACK has 4 divisions', 'NJACK is a technical club', 'NJACK has sub coordinators for every division ', 'NJACK has cp,ML,devos and cybersec divsions']


In [108]:
def generateText(query,tokenizer,model):
    inputs = tokenizer.encode(f"{query}",return_tensors='pt',truncation=True)

    outputs = model.generate(inputs,max_length =100)#Get our outputs

    #Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



In [109]:
#Trying it out now

query = "Tell me about NJACK"

answer = generateText(query, tokenizer, model)

print(f"Answer: {answer}")


Answer:  Tell me about NJACK. Tell us about your favorite NJACK stories. Share your photos of NJACK with iReporter@mailonlinepictures.com. Visit http://www.napp.com/NJACK for more information about the NJACK story.


In [110]:
def generateTextRAG(query,retrieved,tokenizer,model):
    context = " ".join(retrieved)
    inputs = tokenizer.encode(f"{query} {context}",return_tensors='pt',truncation=True)

    outputs = model.generate(inputs,max_length =500)#Get our outputs

    #Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [111]:
documents = pipeline.retrieve('NJACK')
answer2 = generateTextRAG(query,documents, tokenizer, model)

print(f"Answer: {answer2}")

Answer:  NJACK is the coding club of IIT Patna. NJACK has 4 divisions and has sub coordinators for every division. The club is a technical club and technical club. It is also a coding club and has a sub-coordinator for each division of each division.


#Comparing the results

In [112]:
print(answer)
print()
print(answer2)

 Tell me about NJACK. Tell us about your favorite NJACK stories. Share your photos of NJACK with iReporter@mailonlinepictures.com. Visit http://www.napp.com/NJACK for more information about the NJACK story.

 NJACK is the coding club of IIT Patna. NJACK has 4 divisions and has sub coordinators for every division. The club is a technical club and technical club. It is also a coding club and has a sub-coordinator for each division of each division.


We notice that after implementing a very basic RAG, our model performs much better based on our corpus of information. This can be used to improve existing models

## Using CrewAI API